<a href="https://colab.research.google.com/github/yufang934/line_chatbot/blob/main/line_chatbot_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, abort
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import (MessageEvent, TextMessage, TextSendMessage,)
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
import os
from linebot.models import RichMenu
import requests
import random

In [ ]:
#伺服器準備
app = Flask(__name__)

line_bot_api = LineBotApi('Your Channel access token')
handler = WebhookHandler('Your_Channel_secret ')

In [ ]:
#當line封包來，做驗證
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

In [ ]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage, TextSendMessage, ImageSendMessage, LocationSendMessage, FlexSendMessage, VideoSendMessage, StickerSendMessage, AudioSendMessage
)
from linebot.models.template import (
    ButtonsTemplate, CarouselTemplate, ConfirmTemplate, ImageCarouselTemplate
)

from linebot.models.template import *
import json

def detect_json_array_to_new_message_array(fileName):
    # 開啟檔案，轉成json
    with open(fileName,encoding="utf8") as f:
        jsonArray = json.load(f)
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:
        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')

        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'video':
            returnArray.append(VideoSendMessage.new_from_json_dict(jsonObject))

            # 回傳
    return returnArray

In [ ]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''


@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    #判斷是否為json檔，如果是就解讀取並回傳檔案內容；若不是，則打開txt檔
    A=os.path.exists("material/"+event.message.text+"/reply.json")
    B=os.path.exists("material/12stars/"+event.message.text+"/reply.txt")
    C=os.path.exists("material/"+event.message.text+"/reply.txt")
    D=os.path.exists("material/今日穿搭推薦/"+event.message.text+"/1.txt")
    if A==True:
        # 讀取json檔並轉換
        result_message_array = []
        replyJsonPath = "material/"+event.message.text+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

        # 消息發送
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    # 星座運勢：當用戶點選某星座，將資料爬下來並存入檔案中
    elif B==True:
        a=event.message.text
        f = open("material/12stars/" + event.message.text + "/reply.txt","w", encoding="utf-8")
        f.write(str(star_crawler_Capricorn(a)))
        f.close()
        readd=open("material/12stars/" + event.message.text + "/reply.txt","r", encoding="utf-8")
        lines = readd.read()
        readd.close()
        # 消息發送
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(lines)
        )
    #負能量語錄
    elif C==True:
        f = open("material/" + event.message.text + "/reply.txt", encoding="utf-8")
        line = f.readlines()
        randomint=random.randint(0, 48)
        random_line = line[randomint]
        # 消息發送
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(random_line)
        )
    #穿搭推薦
    elif D==True:
        f=open("material/今日穿搭推薦/"+event.message.text+"/1.txt")
        line=f.readlines()
        randomint=random.randint(1,280)
        random_line=line[randomint]
        #消息發送
        #需要圖片的url
        image_message = ImageSendMessage(
            original_content_url=random_line,
            preview_image_url=random_line
        )

        line_bot_api.reply_message(
            event.reply_token,
            image_message
        )


In [ ]:
'''
用戶關注

引用關注事件的套件
用戶關注，發送圖片和訊息
'''
from linebot.models import FollowEvent

@handler.add(FollowEvent)
def handle_follow_event(event):

    # 取出消息內User的資料
    profile = line_bot_api.get_profile(event.source.user_id)
    # 選單功能json檔載入
    menuJson = json.load(open('material/menu/menu.json', 'r', encoding='utf-8'))
    lineRichMenuId = line_bot_api.create_rich_menu(rich_menu=RichMenu.new_from_json_dict(menuJson))
    print(lineRichMenuId)
    # 選單圖片
    uploadImageFile = open('images/menu.png', 'rb')
    setImageResponse = line_bot_api.set_rich_menu_image(lineRichMenuId, 'image/png', uploadImageFile)
    # 將圖片綁定給用戶
    linkResult = line_bot_api.link_rich_menu_to_user(profile.user_id, lineRichMenuId)

In [ ]:
#星座運勢爬蟲
def star_crawler_Capricorn(a):
    stardict = {"Aries": "牡羊座", "Leo": "獅子座", "Sagittarius": "射手座", "Taurus": "金牛座", "Virgo": "處女座", "Capricorn": "摩羯座",
                "Gemini": "雙子座", "Libra": "天秤座", "Aquarius": "水瓶座", "Cancer": "巨蠍座", "Scorpio": "天蠍座", "Pisces": "雙魚座"}
    stardict1 = {"牡羊座": "Aries", "獅子座": "Leo", "射手座": "Sagittarius", "金牛座": "Taurus", "處女座": "Virgo", "摩羯座": "Capricorn",
                "雙子座": "Gemini", "天秤座": "Libra", "水瓶座": "Aquarius", "巨蠍座": "Cancer", "天蠍座": "Scorpio", "雙魚座": "Pisces"}
    starlist = ["Aries", "Leo", "Sagittarius", "Taurus", "Virgo", "Capricorn", "Gemini", "Libra", "Aquarius", "Cancer",
                "Scorpio", "Pisces"]
    url = "https://www.daily-zodiac.com/mobile/zodiac/"+stardict1[a]
    response = urlopen(url)
    html = BeautifulSoup(response)
    r = html.find("div", class_="text")
    contents = r.find("article")
    con = contents.text.strip()
    all = a + ":" + con
    return all

In [ ]:
if __name__ == "__main__":
    #app.run()
    app.run(host='0.0.0.0',port=os.environ['PORT'])